In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px

df= pd.read_csv(r'C:\Users\phuaj\Documents\GitHub\CZ-3006-Labs\cz3006lab4\test_SFLow_data.csv', index_col=False,names=['type','flow_agent_addr',
'inputPort','outputPort','src_MAC','dst_MAC','eth_type','in_vlan','out_vlan',
'src_IP','dst_IP','IP_Protocol','ip_tos','ip_ttl','src_port','dst_port',
'tcp_flags','packet_size','IP_size','sampling_rate'])

df.head()


ModuleNotFoundError: No module named '_plotly_utils'

In [ ]:
ip_lookup = pd.read_csv(r"C:\Users\phuaj\Documents\GitHub\CZ-3006-Labs\cz3006lab4\ip_lookup.csv",engine='python')
ip_lookup.set_index('ip',inplace=True)
ip_lookup.head()

# Frequency Analysis

## Top 5 Talkers

In [ ]:
top5_talkers_ip = pd.DataFrame(df['src_IP'].value_counts()[:5])
top5_talkers_ip= top5_talkers_ip.reset_index()
top5_talkers_ip.columns = ['ip','count']
top5_talkers_ip

In [ ]:
orgs = []
for ip in top5_talkers_ip['ip']:
    try:
        orgs.append(ip_lookup.at[ip,'org'])
    except:
        orgs.append(None)
top5_talkers_ip['org'] = orgs
top5_talkers_ip

In [ ]:
fig = px.pie(top5_talkers_ip, values='count', names='org', title='Traffic Share Among Top 5 Talkers')
fig.show()

## Top 5 Listeners

In [ ]:
top5_listeners_ip = pd.DataFrame(df['dst_IP'].value_counts()[:5])
top5_listeners_ip = top5_listeners_ip.reset_index()
top5_listeners_ip.columns = ['ip','count']
top5_listeners_ip

In [ ]:
orgs = []
for ip in top5_listeners_ip['ip']:
    try:
        orgs.append(ip_lookup.at[ip,'org'])
    except:
        orgs.append(None)
top5_listeners_ip['org'] = orgs
top5_listeners_ip

In [ ]:
fig = px.pie(top5_listeners_ip, values='count', names='org', title='Traffic Share Among Top 5 Talkers')
fig.show()

## Top 3 IP Protocols

In [ ]:
top3_IP_Protocols = pd.DataFrame(df['IP_Protocol'].value_counts()[:3])
top3_IP_Protocols = top3_IP_Protocols.reset_index()
top3_IP_Protocols.columns = ['protocol id','count']
top3_IP_Protocols

In [ ]:
import socket
prefix = "IPPROTO_"
ip_protocol_name = {num:name[len(prefix):] 
          for name,num in vars(socket).items()
            if name.startswith(prefix)}

protocol_names =[]
for pid in top3_IP_Protocols['protocol id'].tolist():
    if pid in ip_protocol_name.keys():
        protocol_names.append(str(pid)+' ({})'.format(ip_protocol_name[pid]))
    else:
        protocol_names.append(str(pid)+' (Unknown Protocol)')

top3_IP_Protocols['Protocol'] = protocol_names

top3_IP_Protocols


In [ ]:
fig = px.pie(top3_IP_Protocols, values='count', names='Protocol', title='Traffic Share Among Top 3 IP Protocols')
fig.show()

In [ ]:
apps = []
for index, row in df.iterrows():
    if df.at[index,'IP_Protocol']==6:
        protocol = 'tcp'
    elif df.at[index,'IP_Protocol']==17:
        protocol = 'udp'
    else:
        protocol = None
    try:
        app = socket.getservbyport(df.at[index,'outputPort'],protocol)+' ({})'.format(str(df.at[index,'outputPort']))
    except:
        app = None
    apps.append(app)

df['app'] = apps
df.head()

In [ ]:
top5_apps_protocol = df['app'].value_counts(dropna=True)[:5]
top5_apps_protocol = top5_apps_protocol.reset_index()
top5_apps_protocol.columns = ['app','count']
top5_apps_protocol

In [ ]:
fig = px.pie(top5_apps_protocol, values='count', names='app', title='Traffic Share Among Top 5 Application Protocols')
fig.show()

## Top 5 Communication Pairs

In [ ]:
pairs={}
for index, row in df.iterrows():
    word1 = row['src_IP']+'/'+row['dst_IP']
    word2 = row['dst_IP']+'/'+row['src_IP']
    if word1 in pairs.keys():
        pairs[word1]+=1
    elif word2 in pairs.keys():
        pairs[word2]+=1
    else:
        pairs[word1]=1

pairs_sorted = sorted([(k,v) for k,v in pairs.items()], key= lambda x: x[1], reverse=True)

pairs_sorted = pd.DataFrame(pairs_sorted[:5])
pairs_sorted.columns = ['pair','count']
pairs_sorted

In [ ]:
org_pairs = []
for pair in pairs_sorted['pair']:
    ips = pair.split('/')
    orgs = []
    for ip in ips:
        try:
            orgs.append(ip_lookup.at[ip,'org'])
        except:
            orgs.append(None)
    org_pairs.append('/'.join(orgs))
pairs_sorted['org'] = org_pairs
pairs_sorted

In [ ]:
fig = px.pie(pairs_sorted, values='count', names='org', title='Traffic Share Among Top 5 Communication Pairs')
fig.show()